In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import math
import time

In [2]:
torch.cuda.is_available()
torch.cuda.FloatTensor()

tensor([], device='cuda:0')

In [12]:
# define dataset
class BowlingDataset(Dataset):
    def __init__(self, fileName):
        self.f = open(fileName, "rb")
        self.length = int(os.stat(fileName).st_size/27)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        self.f.seek(idx)
        gameData = self.f.read(27)
        tempArray = []
        for x in range(25):
            tempArray += [v for v in format(gameData[x], "08b")]
        finalScore = gameData[-1] * 256 + gameData[-2]
        inputArray = [float(v) for v in tempArray][:120]
        output = finalScore
        return torch.tensor(inputArray[:]), torch.tensor(float(output))

In [4]:
""" #this works significantly better but we are hard limited by ram. maybe try to look into memory pinning more
class BowlingDataset(Dataset):
    def __init__(self, fileName):
        self.f = open(fileName, "rb")
        self.length = int(os.stat(fileName).st_size/27)
        if self.length > 1500000:
            self.length = 1500000
        self.f.seek(0)
        # build entire array into memory
        self.inputArray=[]
        self.finalScoreArray=[]
        for v in range(self.length):
            gameData = self.f.read(27)
            tempArray = []
            for x in range(25):
                tempArray += [v for v in format(gameData[x], "08b")]
            self.finalScoreArray.append(gameData[-1] * 256 + gameData[-2])
            self.inputArray.append([float(v) for v in tempArray][:120])
            if not v % 10000:
                print(f"loaded {v} our of {self.length}")
        self.inputTensor = torch.tensor(self.inputArray).to("cuda")
        self.outputTensor = torch.tensor(self.finalScoreArray).to("cuda")
        self.f.close()
    def __len__(self):
        return self.length
    def __getitem__(self, idx):
        return self.inputTensor[idx], self.outputTensor[idx]
"""

'\nclass BowlingDataset(Dataset):\n    def __init__(self, fileName):\n        self.f = open(fileName, "rb")\n        self.length = int(os.stat(fileName).st_size/27)\n        if self.length > 1500000:\n            self.length = 1500000\n        self.f.seek(0)\n        # build entire array into memory\n        self.inputArray=[]\n        self.finalScoreArray=[]\n        for v in range(self.length):\n            gameData = self.f.read(27)\n            tempArray = []\n            for x in range(25):\n                tempArray += [v for v in format(gameData[x], "08b")]\n            self.finalScoreArray.append(gameData[-1] * 256 + gameData[-2])\n            self.inputArray.append([float(v) for v in tempArray][:120])\n            if not v % 10000:\n                print(f"loaded {v} our of {self.length}")\n        self.inputTensor = torch.tensor(self.inputArray).to("cuda")\n        self.outputTensor = torch.tensor(self.finalScoreArray).to("cuda")\n        self.f.close()\n    def __len__(self)

In [5]:
class TestModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.relu_stack = torch.nn.Sequential(
            torch.nn.Linear(120, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 1),
        )
        
    def forward(self, x):
        logits = self.relu_stack(x)
        output = torch.nn.Sigmoid()(logits)
        return output * 300

In [ ]:
model(trainData[0][0])

In [ ]:
trainData[0][0]

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # get number of samples
    totalBatches = len(dataloader)
    model.train() # need to look into what this exactly does
    startTime = time.time()
    for batchNum, (x, y) in enumerate(dataloader):
        # zero out gradients
        optimizer.zero_grad()
        
        # prediction + loss
        prediction = model(x.cuda()).squeeze(1)
        loss = loss_fn(prediction, y.cuda())
        
        #Backpropagation
        loss.backward()
        optimizer.step()
        
        if not batchNum % 1000:
            print(f"loss: {loss.item():.2f}\tbatch num: {batchNum}/{totalBatches}")
            print(f"took {time.time() - startTime} seconds")
            startTime = time.time()

In [7]:
def test_loop(dataloader, model):
    model.eval() # need to look into what this does
    size = len(dataloader.dataset)
    numBatches = len(dataloader)
    test_loss = 0
    
    with torch.no_grad():
        for x, y in dataloader:
            pred = model(x.cuda())
            test_loss += loss_fn(pred, y.cuda()).item()
    
    print(f"Average Loss: {test_loss/numBatches}")

In [8]:
# HYPERPARAMS
batch_size = 64
learning_rate = 0.001
epochs = 8

In [13]:
trainData = BowlingDataset("ScoreDetailDataset.txt")
trainDataLoader = DataLoader(trainData, batch_size=batch_size, shuffle=True) #pin memory doesnt do shit bc the memory has to be grabbed from a physical file
testData = BowlingDataset("ScoreDetailDatasetVSplit.txt")
testDataLoader = DataLoader(testData, batch_size=batch_size, shuffle=True)

In [14]:
# test to see if pinning is working
for batch_ndx, sample in enumerate(trainDataLoader):
    if batch_ndx > 5:
        break
    print(sample[1].is_pinned())
    print(sample[1].is_cuda)


RuntimeError: DataLoader worker (pid(s) 26840, 18000, 28616, 21936, 14764, 13224, 15052, 10572, 27912, 28364, 27032, 21580) exited unexpectedly

In [11]:
model = TestModel().cuda()
loss_fn = torch.nn.L1Loss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(epochs):
    startTime = time.time()
    print(f"Starting epoch {t}")
    train_loop(trainDataLoader, model, loss_fn, optimizer)
    print(f"Epoch {t} took {time.time() - startTime} seconds")
    test_loop(testDataLoader, model)
print("Finished")

Starting epoch 0


TypeError: cannot pickle '_io.BufferedReader' object

In [ ]:
trainDataLoader = DataLoader(trainData, batch_size=batch_size, shuffle=True, pin_memory=False)